# 📝 License and Notices

This notebook is part of "Arduino 2D Pan-Tilt Control System + AI Mosquito Auto-Tracking" (mosquito-pt2d).

Copyright 2025 Arduino PT2D Project

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this work except in compliance with the License. You may obtain a copy of the License at:

- http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

For project and third-party notices, see the repository files:
- LICENSE
- NOTICE

## 1. 掛載 Google Drive

## 📋 完整工作流程總覽

### 階段 1: 本機標註與準備 (label_samples.py)

使用 `label_samples.py` 標註樣本，搬遷時自動完成：
1. 創建訓練數據集結構 (training_dataset/)
2. 進行 80/20 隨機分割
3. 驗證/校正所有標籤類別 ID
4. 生成 dataset.yaml
5. 備份原始樣本

```bash
cd d:/Workspaces/mosquito-pt2d/python
python label_samples.py
# 選擇 4. 搬遷已標註樣本
```

**輸出**: `relocated_samples/<TIMESTAMP>/training_dataset/`

### 階段 2: Colab 訓練 (本 Notebook)

| 步驟 | Cell | 輸出 | 說明 |
|------|------|------|------|
| 1. 掛載 Drive | Cell 2 | - | Google Drive 連結 |
| 2. 安裝環境 | Cell 3 | - | ultralytics, torch |
| 3. ~~準備數據集~~ **已移至本機** | ~~Cell 4~~ | - | ✅ label_samples.py 已完成 |
| 3. 訓練模型 | Cell 4 | `runs/detect/mosquito_dual_platform/` | 100 epochs |
| 4. 評估結果 | Cell 5 | mAP50, mAP50-95 | 精度指標 |
| 5. 保存模型 | Cell 6 | `models/mosquito_yolov8_new.pt` | ⚠️ 關鍵檔案 |
| 6. 查看曲線 | Cell 7 | 訓練圖表 | 可視化 |
| 7. 模型轉換 | Cell 8 | `models/*.{onnx,rknn,bin}` | 全格式支援 |

### 階段 3: 本機部署 (deploy_model.py)

```bash
# 前置條件: Google Drive 已同步訓練好的模型
cd d:/Workspaces/mosquito-pt2d
python python/deploy_model.py
```

| 步驟 | 動作 | 輸出 | 說明 |
|------|------|------|------|
| 1. 備份 | 自動 | `mosquito_yolov8_backup_*.pt` | 保護舊模型 |
| 2. 複製 | 自動 | `models/mosquito_yolov8.pt` | 從 Google Drive |
| 3. ONNX | 自動 | `models/mosquito_yolov8.onnx` | 兩平台共用 |
| 4. RKNN | 自動 | `models/mosquito_yolov8.rknn` | Orange Pi 5 |
| 5. RDK X5 | 手動 | `mosquito_yolov8.bin` | 需 hb_mapper |

### 階段 4: 平台部署

**Orange Pi 5** (自動完成):
```bash
python python/streaming_tracking_system.py
```

**RDK X5** (需額外步驟):
```bash
# 1. x86 機器編譯 .bin
python3 mapper.py --onnx models/mosquito_yolov8.onnx --cal-images calibration_images

# 2. 複製到板端
scp mosquito_yolov8.bin rdkx5:/path/to/models/

# 3. 運行追蹤系統
python python/streaming_tracking_system.py
```

### 🔗 關鍵路徑對應

✅ **本機標註 → 訓練數據集**
- label_samples.py 搬遷時自動生成完整結構
- 本機輸出: `relocated_samples/<TIMESTAMP>/training_dataset/`
- 手動複製到: `D:\Users\jeffma\Sync\Google\我的雲端硬碟\Colab Notebooks\mosquito-training\relocated_samples\`

✅ **Colab Google Drive 路徑對應**
- 本機: `D:\Users\jeffma\Sync\Google\我的雲端硬碟\Colab Notebooks\`
- Colab: `/content/drive/MyDrive/Colab Notebooks/`
- 訓練數據集: `/content/drive/MyDrive/Colab Notebooks/mosquito-training/relocated_samples/<TIMESTAMP>/training_dataset/`

✅ **Colab 訓練 → Google Drive**
- 輸出路徑: `/content/drive/MyDrive/Colab Notebooks/mosquito-training/models/mosquito_yolov8_new.pt`
- 本機同步到: `D:\Users\jeffma\Sync\Google\我的雲端硬碟\Colab Notebooks\mosquito-training\models\`

✅ **Google Drive → 本機專案**
- deploy_model.py 自動推導路徑（基於 config.COLAB_NOTEBOOK_DEST_DIR）
- 複製到: `d:\Workspaces\mosquito-pt2d\models\`

✅ **本機 → 目標板**
- Orange Pi 5: 直接使用 `models/mosquito_yolov8.rknn`
- RDK X5: 手動複製 `mosquito_yolov8.bin`

### 🚀 一鍵命令

```bash
# 完整流程（Orange Pi 5）
python python/deploy_model.py && python python/streaming_tracking_system.py

# 僅更新模型
python python/deploy_model.py
```

In [ ]:
from google.colab import drive
from pathlib import Path
import shutil
import os

drive.mount('/content/drive')

# 設定工作目錄
colab_base = '/content/drive/MyDrive/Colab Notebooks/mosquito-training'
os.makedirs(colab_base, exist_ok=True)
os.chdir(colab_base)

print("✓ Google Drive 已掛載")
print(f"✓ 工作目錄: {os.getcwd()}")

# 檢查訓練數據集是否存在
training_dataset_base = Path(colab_base) / 'relocated_samples'
training_dataset_dirs = sorted([d for d in training_dataset_base.glob('*') if d.is_dir()]) if training_dataset_base.exists() else []

if training_dataset_dirs:
    latest_timestamp = training_dataset_dirs[-1]
    training_dataset = latest_timestamp / 'training_dataset'

    if training_dataset.exists():
        # 複製訓練數據集到工作目錄
        local_training_dataset = Path(colab_base) / 'training_dataset'
        if local_training_dataset.exists():
            shutil.rmtree(local_training_dataset)

        shutil.copytree(training_dataset, local_training_dataset)
        print(f"\n✓ 訓練數據集已複製到工作目錄")
        print(f"  來源: {training_dataset}")
        print(f"  目標: {local_training_dataset}")

        # 驗證 dataset.yaml
        dataset_yaml = local_training_dataset / 'dataset.yaml'
        if dataset_yaml.exists():
            print(f"\n✓ dataset.yaml 已正確複製")
            print(f"  位置: {dataset_yaml}")
        else:
            raise FileNotFoundError(f"❌ 錯誤: dataset.yaml 未複製\n   預期位置: {dataset_yaml}")

        # 驗證數據集結構
        train_imgs = list((local_training_dataset / 'images' / 'train').glob('*.jpg'))
        val_imgs = list((local_training_dataset / 'images' / 'val').glob('*.jpg'))

        # 驗證標籤檔
        train_labels = list((local_training_dataset / 'labels' / 'train').glob('*.txt'))
        val_labels = list((local_training_dataset / 'labels' / 'val').glob('*.txt'))

        print(f"\n✓ 數據集統計:")
        print(f"  訓練集圖片: {len(train_imgs)} 張, 標籤: {len(train_labels)} 個")
        print(f"  驗證集圖片: {len(val_imgs)} 張, 標籤: {len(val_labels)} 個")

        if len(train_imgs) != len(train_labels) or len(val_imgs) != len(val_labels):
            raise ValueError("⚠️ 警告: 圖片和標籤數量不匹配!")
    else:
        print(f"⚠️ 警告: 找不到 training_dataset 目錄")
        print(f"   預期路徑: {training_dataset}")
else:
    print("⚠️ 警告: 找不到 relocated_samples 目錄")
    print("   請確認已在本機執行 label_samples.py 並搬遷樣本到 Google Drive")

## 2. 安裝訓練環境

In [ ]:
!pip install ultralytics -q

# 檢查 GPU
import torch
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ 警告：未偵測到 GPU，請檢查執行階段設定")

## 3. 開始訓練模型

In [ ]:
from ultralytics import YOLO

# 載入預訓練模型
# 建議：使用 YOLOv8n（nano）以獲得最佳性能/精度平衡
model = YOLO('yolov8n.pt')  # 自動下載預訓練模型

# 訓練參數（適配 RDK X5 與 Orange Pi 5）
# RDK X5 參考: https://github.com/D-Robotics/rdk_model_zoo
# Orange Pi 5 參考: https://github.com/airockchip/rknn-toolkit2
results = model.train(
    data='training_dataset/dataset.yaml',
    epochs=100,            # 充足訓練以提升量化後精度
    imgsz=640,             # 兩平台標準輸入尺寸（YOLOv8n: 640×640）
    batch=16,              # 根據 GPU 記憶體調整
    lr0=0.01,              # 學習率（YOLOv8 預設）
    name='mosquito_dual_platform',
    patience=20,           # 增加 patience 避免過早停止
    device=0,              # 使用 GPU
    workers=4,             # 資料載入執行緒
    amp=True,              # 自動混合精度訓練（加速）
    # 量化友好設定
    close_mosaic=10,       # 最後 10 個 epoch 關閉 Mosaic 增強
    optimizer='AdamW',     # 優化器
    cos_lr=True            # 餘弦學習率衰減
)

print("\n" + "="*50)
print("✓ 訓練完成!")
print(f"✓ 最佳模型: runs/detect/mosquito_dual_platform/weights/best.pt")
print("="*50)

## 4. 評估訓練結果

In [ ]:
# 驗證模型
metrics = model.val()

print("\n訓練指標：")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"精確率: {metrics.box.mp:.4f}")
print(f"召回率: {metrics.box.mr:.4f}")

## 5. (可選) 查看訓練曲線

In [ ]:
from IPython.display import Image, display

# 顯示訓練結果圖表
print("訓練曲線：")
display(Image('runs/detect/mosquito_dual_platform/results.png'))

print("\n混淆矩陣：")
display(Image('runs/detect/mosquito_dual_platform/confusion_matrix.png'))

## 6. 模型格式轉換（全平台一鍵生成）

將訓練好的 PyTorch 模型轉換為所有部署格式：
- **ONNX**: 通用格式
- **RKNN**: Orange Pi 5 (RK3588 NPU) 專用格式
- **BIN**: RDK X5 (Bayes-e BPU) 專用格式

完成後，`deploy_model.py` 只需複製模型文件即可。

In [ ]:
# 安裝模型轉換工具（僅 Orange Pi 5 RKNN）
print("📦 安裝模型轉換工具...")

import platform
is_x86 = platform.machine() == 'x86_64'

if not is_x86:
    print("⚠️ 警告: 需要 x86_64 架構，當前架構:", platform.machine())
    print("   僅能生成 ONNX 模型")
else:
    # 安裝基礎工具
    print("安裝 ONNX 相關工具...")
    !pip install onnx onnxsim --quiet

    # 安裝 rknn-toolkit2（Orange Pi 5）
    print("安裝 rknn-toolkit2 (Orange Pi 5)...")
    !pip install rknn-toolkit2 -i https://pypi.tuna.tsinghua.edu.cn/simple --quiet

    print("✅ 工具鏈安裝完成")

    # 驗證 rknn-toolkit2
    try:
        from rknn.api import RKNN
        print("✅ rknn-toolkit2 可用")
    except ImportError:
        print("⚠️ rknn-toolkit2 導入失敗")

# RDK X5 相關工具已註解（需要時再啟用）
# print("安裝 hb_mapper (RDK X5)...")
# !pip install horizon-tc-ui -i https://pypi.tuna.tsinghua.edu.cn/simple --quiet
# !hb_mapper --version

In [ ]:
# 準備校準數據集（用於 BIN 模型量化）
import shutil
import random
from pathlib import Path

colab_base = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training')
calib_dir = colab_base / 'calibration_images'
calib_dir.mkdir(parents=True, exist_ok=True)

# 從訓練集隨機抽取 50 張作為校準數據
training_dataset = colab_base / 'training_dataset'
train_images = list((training_dataset / 'images' / 'train').glob('*.jpg'))

if len(train_images) < 10:
    raise ValueError(f"訓練集圖片不足（僅 {len(train_images)} 張），至少需要 10 張用於校準")

random.seed(42)
calib_samples = random.sample(train_images, min(50, len(train_images)))

print(f"📸 準備 {len(calib_samples)} 張校準圖片...")
for img in calib_samples:
    shutil.copy(img, calib_dir / img.name)

print(f"   圖片數量: {len(list(calib_dir.glob('*.jpg')))}")
print(f"✅ 校準數據集準備完成: {calib_dir}")

In [ ]:
# 導出 ONNX 模型（通用格式）
from ultralytics import YOLO
from pathlib import Path
import shutil
from datetime import datetime
import onnx
from onnxsim import simplify

# 設定路徑
model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
model_dir.mkdir(parents=True, exist_ok=True)

# 使用訓練輸出的最佳模型
best_pt = Path('runs/detect/mosquito_dual_platform/weights/best.pt')

if not best_pt.exists():
    raise FileNotFoundError(f"找不到訓練輸出的 best.pt，預期路徑: {best_pt}")

# 檔案存在，繼續處理
if True:
    # 1. 先保存 PyTorch 模型到 Google Drive
    print("📦 保存 PyTorch 模型...")
    pt_save_path = model_dir / 'mosquito_yolov8_new.pt'
    shutil.copy(best_pt, pt_save_path)
    print(f"✅ PyTorch 模型已保存: {pt_save_path}")

    # 同時備份一份帶日期的版本
    date_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_path = model_dir / f'mosquito_yolov8_backup_{date_str}.pt'
    shutil.copy(best_pt, backup_path)
    print(f"✅ 備份版本已保存: {backup_path}")

    # 2. 導出 ONNX（不簡化）
    print("\n📦 導出 ONNX 模型...")
    model = YOLO(str(best_pt))
    model.export(format='onnx', imgsz=640, opset=12, simplify=False)

    # 將導出的 ONNX 移動到臨時位置
    onnx_path = Path('runs/detect/mosquito_dual_platform/weights/best.onnx')
    if not onnx_path.exists():
        raise FileNotFoundError(f"ONNX 導出失敗，找不到: {onnx_path}")

    # 3. 使用 onnxsim 簡化模型（RDK X5 hb_mapper 必須）
    print("🔧 使用 onnxsim 簡化 ONNX 模型（RDK X5 BIN 編譯必要步驟）...")
    onnx_model = onnx.load(str(onnx_path))
    model_simplified, check = simplify(onnx_model)

    if not check:
        raise RuntimeError(
            "ONNX 簡化失敗！這是 RDK X5 BIN 模型編譯的必要步驟。\n"
            "可能原因：\n"
            "  1. ONNX 模型結構不符合 onnxsim 要求\n"
            "  2. onnxsim 版本不兼容（需要 0.4.33）\n"
            "  3. ONNX 模型本身有問題"
        )

    # 簡化成功，保存模型
    onnx_save_path = model_dir / 'mosquito_yolov8.onnx'
    onnx.save(model_simplified, str(onnx_save_path))
    print(f"✅ ONNX 模型已簡化並保存: {onnx_save_path}")

    # 顯示模型大小
    size_mb = onnx_save_path.stat().st_size / 1024 / 1024
    print(f"   文件大小: {size_mb:.2f} MB")

In [ ]:
# ==================== RDK X5 BIN 模型編譯（已註解） ====================
# 創建 hb_mapper 配置文件
# from pathlib import Path
#
# model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
# calib_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/calibration_images')
# onnx_model = model_dir / 'mosquito_yolov8.onnx'
# config_path = Path('/content/yolov8_config.yaml')
#
# config_content = f"""model_parameters:
#   onnx_model: '{str(onnx_model)}'
#   march: 'bayes-e'
#   layer_out_dump: False
#   working_dir: '/content/model_output'
#   output_model_file_prefix: 'mosquito_yolov8'
#
# input_parameters:
#   input_name: 'images'
#   input_type_rt: 'nv12'
#   input_layout_rt: 'NHWC'
#   input_type_train: 'rgb'
#   input_layout_train: 'NCHW'
#   norm_type: 'data_scale'
#   scale_value: 0.003921568627451
#   input_shape: '1x3x640x640'
#
# calibration_parameters:
#   cal_data_dir: '{str(calib_dir)}'
#   cal_data_type: 'float32'
#   calibration_type: 'default'
#
# compiler_parameters:
#   compile_mode: 'latency'
#   debug: False
#   optimize_level: 'O3'
# """
#
# config_path.write_text(config_content)
# if not config_path.exists():
#     raise IOError(f"配置文件創建失敗: {config_path}")
# else:
#     print(f"✅ 配置文件已創建: {config_path}")

print("⚠️ RDK X5 BIN 模型編譯已跳過（配置文件創建已註解）")

In [ ]:
# ==================== RDK X5 BIN 模型編譯（已註解） ====================
# 執行 hb_mapper 編譯（RDK X5 BIN 格式）
# import platform
# from pathlib import Path
# import shutil
#
# model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
#
# if platform.machine() != 'x86_64':
#     raise RuntimeError(
#         f"環境架構錯誤：當前架構為 {platform.machine()}，但 RDK X5 BIN 模型編譯需要 x86_64。\n"
#         "Colab 預設應為 x86_64，請檢查：\n"
#         "  1. Colab 執行階段設定\n"
#         "  2. 是否使用了非標準的執行環境\n"
#         "  3. 重新啟動執行階段後再試"
#     )
#
# print("🔧 開始編譯 RDK X5 BIN 模型...")
# print("⏱️  預計耗時: 5-10 分鐘\n")
#
# !hb_mapper makertbin --config /content/yolov8_config.yaml
#
# # 檢查輸出
# bin_model = Path('/content/model_output/mosquito_yolov8.bin')
# if not bin_model.exists():
#     raise RuntimeError("BIN 模型編譯失敗，請檢查上方錯誤信息")
#
# print("\n✅ BIN 模型編譯成功!")
# print(f"   輸出路徑: {bin_model}")
# print(f"   文件大小: {bin_model.stat().st_size / 1024 / 1024:.2f} MB")
#
# # 複製到 models 目錄
# output_bin = model_dir / 'mosquito_yolov8.bin'
# shutil.copy(bin_model, output_bin)
# print(f"   已複製到: {output_bin}")

print("⚠️ RDK X5 BIN 模型編譯已跳過")

In [ ]:
# 生成 RKNN 模型（Orange Pi 5）
import platform
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
onnx_model = model_dir / 'mosquito_yolov8.onnx'
calib_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/calibration_images')

if platform.machine() != 'x86_64':
    raise RuntimeError(
        f"環境架構錯誤：當前架構為 {platform.machine()}，但 RKNN 模型生成需要 x86_64。\n"
        "Colab 預設應為 x86_64，請檢查：\n"
        "  1. Colab 執行階段設定\n"
        "  2. 是否使用了非標準的執行環境\n"
        "  3. 重新啟動執行階段後再試"
    )

print("🔧 開始生成 Orange Pi 5 RKNN 模型...")
print("⏱️  預計耗時: 3-5 分鐘\n")

try:
    from rknn.api import RKNN

    # 初始化 RKNN 物件
    rknn = RKNN(verbose=False)

    # 配置模型
    print("📝 配置 RKNN 參數...")
    rknn.config(
        mean_values=[[0, 0, 0]],
        std_values=[[255, 255, 255]],
        target_platform='rk3588'
    )

    # 載入 ONNX 模型
    print("📦 載入 ONNX 模型...")
    ret = rknn.load_onnx(model=str(onnx_model))
    if ret != 0:
        raise RuntimeError('載入 ONNX 模型失敗!')

    # 使用校準數據集進行量化
    print("⚙️  執行量化（使用校準數據集）...")
    ret = rknn.build(
        do_quantization=True,
        dataset=str(calib_dir)
    )

    if ret != 0:
        raise RuntimeError('RKNN build 失敗!')

    # 導出 RKNN 模型
    rknn_file = model_dir / 'mosquito_yolov8.rknn'
    ret = rknn.export_rknn(str(rknn_file))

    if ret != 0:
        raise RuntimeError('導出 RKNN 模型失敗!')

    print(f"\n✅ RKNN 模型生成成功!")
    print(f"   輸出路徑: {rknn_file}")
    print(f"   文件大小: {rknn_file.stat().st_size / 1024 / 1024:.2f} MB")

    # 釋放資源
    rknn.release()

except ImportError as e:
    raise ImportError(
        f"rknn-toolkit2 導入失敗: {e}\n"
        "請確認安裝步驟是否正確執行。"
    )
except Exception as e:
    raise RuntimeError(f"RKNN 生成失敗: {e}")

In [ ]:
# 模型轉換完成摘要
import zipfile
from datetime import datetime
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')

print("\n" + "="*50)
print("📊 模型轉換完成")
print("="*50)

# 檢查生成的模型
pt_file = model_dir / 'mosquito_yolov8_new.pt'
onnx_file = model_dir / 'mosquito_yolov8.onnx'
rknn_file = model_dir / 'mosquito_yolov8.rknn'
bin_file = model_dir / 'mosquito_yolov8.bin'

print("\n已生成的模型:")
model_files = []
if pt_file.exists():
    print(f"  ✅ PyTorch: {pt_file.name} ({pt_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(pt_file)
if onnx_file.exists():
    print(f"  ✅ ONNX: {onnx_file.name} ({onnx_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(onnx_file)
if rknn_file.exists():
    print(f"  ✅ RKNN (Orange Pi 5): {rknn_file.name} ({rknn_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(rknn_file)
if bin_file.exists():
    print(f"  ✅ BIN (RDK X5): {bin_file.name} ({bin_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(bin_file)

# 創建帶日期戳記的 ZIP 備份
if model_files:
    print("\n" + "="*50)
    print("📦 創建備份壓縮檔")
    print("="*50)

    date_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = model_dir / f'mosquito_models_backup_{date_str}.zip'

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for model_file in model_files:
            zipf.write(model_file, model_file.name)
            print(f"  ✅ 已加入: {model_file.name}")

    zip_size_mb = zip_filename.stat().st_size / 1024 / 1024
    print(f"\n✅ 備份完成: {zip_filename.name} ({zip_size_mb:.2f} MB)")
    print(f"   位置: {zip_filename}")

print("\n📥 下一步:")
print("  1. 等待 Google Drive 同步完成")
print("  2. 執行 python/deploy_model.py（自動複製所有模型到專案目錄）")
print("  3. 系統會自動選擇對應平台的模型格式:")
print("     - Orange Pi 5 → mosquito_yolov8.rknn")
print("     - RDK X5 → mosquito_yolov8.bin")